<a href="https://colab.research.google.com/github/SJY-coder/TSP-demo/blob/master/TSP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fetch Kaggle Datasets into Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d aaron7sun/stocknews

 86% 5.00M/5.82M [00:00<00:00, 45.9MB/s]
100% 5.82M/5.82M [00:00<00:00, 37.1MB/s]


In [ ]:
!ls

Combined_News_DJIA.csv	kaggle.json  RedditNews.csv  upload_DJIA_table.csv


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  stocknews.zip
  inflating: Combined_News_DJIA.csv  
  inflating: RedditNews.csv          
  inflating: upload_DJIA_table.csv   


In [ ]:
!ls

Combined_News_DJIA.csv	kaggle.json  RedditNews.csv  upload_DJIA_table.csv


#**Preprocessing Datasets**

### merge 'Combined_News_DJIA.csv' with 'upload_DJIA_table.csv' to predict precise open&close price for the next day.


In [ ]:
import pandas as pd
df1 = pd.read_csv('Combined_News_DJIA.csv')
df2 = pd.read_csv('upload_DJIA_table.csv')
#merge on 'Date' column for left&right
full_df = df1.merge(df2, left_on = 'Date', right_on = 'Date')

In [ ]:
print(df1.shape,"+",df2.shape,"=>",full_df.shape)
full_df.head()

(1989, 27) + (1989, 7) => (1989, 33)


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS 

In [ ]:
import numpy as np
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
#giving df a new column of nexts day's open price & close price for target + text preprocessing
str_df = full_df.iloc[:,2:27].astype(str)
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
  """custom function to remove punctuation"""
  return text.translate(str.maketrans('', '',PUNCT_TO_REMOVE))
def remove_b(str_df): 
  for i in range(str_df.shape[1]):
    str_df.iloc[:,i] = str_df.iloc[:,i].apply(lambda text: remove_punctuation(text))
    for j in range(len(str_df.iloc[:,1])):
      #deleting 'b' in front of sentence
      str_df.iloc[:,i][j] = str_df.iloc[:,i][j][1:]
  return str_df
remove_b(str_df)
full_df.iloc[:,2:27] = str_df
full_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity UN officia...,150 Russian tanks have entered South Ossetia w...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,Georgian troops retreat from S Osettain capita...,Did the US Prep Georgia for War with Russia,Rice Gives Green Light for Israel to Attack Ir...,AnnouncingClass Action Lawsuit on Behalf of Am...,SoRussia and Georgia are at war and the NYTs t...,China tells Bush to stay out of other countrie...,Did World War III start today,Georgia Invades South Ossetia if Russia gets ...,AlQaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has app...,Georgia will withdraw 1000 soldiers from Iraq ...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a serie...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexicos Kidnapping Surge,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in SOssetia blames ...,Welcome To World War IV Now In High Definition,Georgias move a mistake of monumental proporti...,Russia presses deeper into Georgia US says reg...,Abhinav Bindra wins first ever Individual Olym...,US ship heads for Arctic to define territory,Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggression,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia PICS,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps the question about the Georgia Russia...,Russia is so much better at war,So this is what its come to trading sex for food,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,Remember that adorable 9yearold who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,AlQaeda is losing support in Iraq because of a...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO 1...,Stratfor The RussoGeorgian War and the Balance...,Im Trying to Get a Sense of This Whole Georgia...,The US military was surprised by the timing an...,US Beats War Drum as Iran Dumps the Dollar,Gorbachev Georgian military attacked the South...,CNN use footage of Tskhinvali ruins to cover G...,Beginning a war as the Olympics were opening v...,55 pyramids as large as the Luxor stacked into...,The 11 Top Party Cities in the World,US troops still in Georgia did you know they w...,Why Russias response to Georgia was right,Gorbachev accuses US of making a serious blund...,Russia Georgia and NATO Cold War Two,Remember that adorable 62yearold who led your ...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS AsiaPacific Extinction by man not c...,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,US refuses Israel weapons to attac

In [ ]:
#create new column with next row's open & close price as target and drop last row
# df = full_df
df_open_tmp = pd.DataFrame(df.iloc[1:,-6])
df_close_tmp = pd.DataFrame(df.iloc[1:,-3])
new_row_1 = pd.DataFrame({"Open": [0]})
new_row_2 = pd.DataFrame({"Close": [0]})
new_row_open = df_open_tmp.append(new_row_1, ignore_index=True)
new_row_close = df_close_tmp.append(new_row_2, ignore_index=True)
df["p-Open"] = new_row_open
df["p-Close"] = new_row_close
df = df.iloc[:-1,:]

In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close,p-Open,p-Close
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity UN officia...,150 Russian tanks have entered South Ossetia w...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,Georgian troops retreat from S Osettain capita...,Did the US Prep Georgia for War with Russia,Rice Gives Green Light for Israel to Attack Ir...,AnnouncingClass Action Lawsuit on Behalf of Am...,SoRussia and Georgia are at war and the NYTs t...,China tells Bush to stay out of other countrie...,Did World War III start today,Georgia Invades South Ossetia if Russia gets ...,AlQaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has app...,Georgia will withdraw 1000 soldiers from Iraq ...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a serie...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexicos Kidnapping Surge,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,11729.669922,11782.349609
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in SOssetia blames ...,Welcome To World War IV Now In High Definition,Georgias move a mistake of monumental proporti...,Russia presses deeper into Georgia US says reg...,Abhinav Bindra wins first ever Individual Olym...,US ship heads for Arctic to define territory,Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggression,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia PICS,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps the question about the Georgia Russia...,Russia is so much better at war,So this is what its come to trading sex for food,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,11781.700195,11642.469727
2,2008-08-12,0,Remember that adorable 9yearold who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,AlQaeda is losing support in Iraq because of a...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO 1...,Stratfor The RussoGeorgian War and the Balance...,Im Trying to Get a Sense of This Whole Georgia...,The US military was surprised by the timing an...,US Beats War Drum as Iran Dumps the Dollar,Gorbachev Georgian military attacked the South...,CNN use footage of Tskhinvali ruins to cover G...,Beginning a war as the Olympics were opening v...,55 pyramids as large as the Luxor stacked into...,The 11 Top Party Cities in the World,US troops still in Georgia did you know they w...,Why Russias response to Georgia was right,Gorbachev accuses US of making a serious blund...,Russia Georgia and NATO Cold War Two,Remember that adorable 62yearold who led your ...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS AsiaPacific Extinction by man not c...,11781.700195,11782.349609,11601.519531,11642.469727,17359

In [ ]:
#spliting sentiment,stock price,target stock price for model
df_num, df_y, df_sen = df.iloc[:,-8:-2],df.iloc[:,-2:], df.iloc[:,2:-8]
df_y.insert(0,'Label',df.iloc[:,1])

print(df_num.head(),df_num.shape,'\n',df_sen.head(),df_sen.shape,'\n',df_y,df_y.shape)


           Open          High  ...     Volume     Adj Close
0  11432.089844  11759.959961  ...  212830000  11734.320312
1  11729.669922  11867.110352  ...  183190000  11782.349609
2  11781.700195  11782.349609  ...  173590000  11642.469727
3  11632.809570  11633.780273  ...  182550000  11532.959961
4  11532.070312  11718.280273  ...  159790000  11615.929688

[5 rows x 6 columns] (1988, 6) 
                                                 Top1  ...                                              Top25
0  Georgia downs two Russian warplanes as countri...  ...               No Help for Mexicos Kidnapping Surge
1  Why wont America and Nato help us If they wont...  ...   So this is what its come to trading sex for food
2  Remember that adorable 9yearold who sang at th...  ...  BBC NEWS  AsiaPacific  Extinction by man not c...
3    US refuses Israel weapons to attack Iran report  ...  2006 Nobel laureate Aleksander Solzhenitsyn ac...
4  All the experts admit that we should legalise ...  ...  Ph